[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1m_8z6ph3-nsp_3NGxU9aHg4dmeLogsfG?usp=sharing)

useful links:
* CIRAF10 example: http://home.mit.bme.hu/~hadhazi/Oktatas/NN18/dem3/html_demo/CIFAR-10Demo.html
* keras tuner: https://github.com/BME-SmartLab-Education/vitmav45/blob/master/12/keras_tuner_fashionmnist.ipynb

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 2.8 MB/s 


In [2]:
import numpy as np  
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical 

# Import the CIRAF10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Transform label indices to one-hot encoded vectors
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Change data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalization of pixel values (to [0-1] range)
x_train /= 255.0
x_test /= 255.0

170508288/170498071 [==============================] - 11s 0us/step


In [3]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping

def create_model(hp):
  # The hyperparameter space:
  layer_size_1 = hp.Choice('layer_size_1', values = [16, 32, 64])
  layer_size_2 = hp.Choice('layer_size_2', values = [32, 64, 128])
  layer_size_3 = hp.Choice('layer_size_3', values = [64, 128, 256])
  fully_connected_size = hp.Choice('fully_connected_size', values = [256, 512, 1024])
  dropout_1 = hp.Float('dropout_1', min_value=0, max_value=0.5)
  dropout_2 = hp.Float('dropout_2', min_value=0, max_value=0.5)
  dropout_3 = hp.Float('dropout_3', min_value=0, max_value=0.5)

  learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  # Create model
  model = Sequential()

  # I will use 2 or 3 2D conv layers, with relu activations, each followed by a a dropout layer, the last one with a maxpooling also
  # layer sizes and dropout rates will be optimized
  # at the end, I add a fully connected layer with variable size
  # all activations will be relu, to change that, use activation=hp.Choice('activation', values = ['relu', 'swish']),
  model.add(Conv2D(
    layer_size_1,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=(32,32,3) )) 
  #model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( dropout_1 ))

  model.add(Conv2D(
    layer_size_1,
    kernel_size=(3, 3),
    activation='relu'))
  #model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( dropout_2 ))

  if(hp.Choice('third_layer', values = [True, False])):
    model.add(Conv2D(
      layer_size_1,
      kernel_size=(3, 3),
      activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout( dropout_2 ))

  model.add(Flatten())
  model.add(Dense(fully_connected_size, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),               
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [4]:
from kerastuner.tuners import Hyperband

# Create a tuner for auto hyperparameter optimaziton
tuner = Hyperband(
    create_model,
    objective='val_accuracy',
    factor=3,
    max_epochs=8,
    hyperband_iterations=1,
    directory='output',
    project_name='CIRAF10_hyperband')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [5]:
# Wiew the hiperparam space
tuner.search_space_summary()

Search space summary
Default search space size: 9
layer_size_1 (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
layer_size_2 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128], 'ordered': True}
layer_size_3 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
fully_connected_size (Choice)
{'default': 256, 'conditions': [], 'values': [256, 512, 1024], 'ordered': True}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': None, 'sampling': None}
dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': None, 'sampling': None}
dropout_3 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': None, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
third_layer (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'o

In [ ]:
# Actual hyperparamatization
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
tuner.search(x_train, y_train, epochs=8,
             validation_split=0.2, callbacks=[early_stopping], shuffle=True)

Trial 9 Complete [00h 02m 23s]
val_accuracy: 0.6385999917984009

Best val_accuracy So Far: 0.6559000015258789
Total elapsed time: 00h 17m 16s

Search: Running Trial #10

Hyperparameter    |Value             |Best Value So Far 
layer_size_1      |16                |16                
layer_size_2      |64                |128               
layer_size_3      |256               |128               
fully_connected...|256               |256               
dropout_1         |0.0021817         |0.32799           
dropout_2         |0.29475           |0.18296           
dropout_3         |0.3258            |0.23719           
learning_rate     |0.0001            |0.001             
third_layer       |1                 |1                 
tuner/epochs      |8                 |8                 
tuner/initial_e...|0                 |3                 
tuner/bracket     |0                 |1                 
tuner/round       |0                 |1                 

Epoch 1/8
1250/1250 [==========

In [ ]:
# Parameters of the best model
params_best = tuner.get_best_hyperparameters(num_trials=1)[0]
params_best.get_config()['values']

In [ ]:
# 10 best training
tuner.results_summary()

In [ ]:
# Get the best model
hypermodel = tuner.hypermodel.build(params_best)

# Retrain the model
history = hypermodel.fit(x_train, y_train, epochs=100, validation_split=0.2, shuffle=True, callbacks = early_stopping, batch_size = 32)

In [ ]:
score = hypermodel.evaluate(x_test, y_test, batch_size=128, verbose=0)
print(hypermodel.metrics_names)
print(score)

In [ ]:
hypermodel.summary()